In [31]:
# Model loading
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
model_path = "../TinyLlama-1.1B-checkpoint"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(model_path, quantization_config=bnb_config, device_map="auto")

In [32]:
# Gradient checkpointing and kbit training
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [33]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [34]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    lora_dropout=0.05, 
    bias="none", 
    task_type="lm",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1126400 || all params: 616736768 || trainable%: 0.18263869748722358


In [35]:
# Make the training dataset from ./data/Q2_iCliniq_data.json. 
from datasets import load_dataset
training_data = load_dataset("json", data_files="./data/Q2_iCliniq_data.json")
training_data = training_data.map(lambda sample: tokenizer(f"### Human: {sample['input']} ### Assistant: {sample['answer_icliniq']}"))

In [36]:
import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset=training_data["train"]["input_ids"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
1,3.052600
2,3.021200
3,3.049100
4,3.067500
5,2.990500
6,3.047900
7,2.886700
8,2.971500
9,2.858800
10,2.879400


TrainOutput(global_step=200, training_loss=2.4387497472763062, metrics={'train_runtime': 923.9106, 'train_samples_per_second': 13.854, 'train_steps_per_second': 0.216, 'total_flos': 4.381966205288448e+16, 'train_loss': 2.4387497472763062, 'epoch': 1.75})

In [37]:
# Save model, config and tokenizer
model.save_pretrained("./outputs")
tokenizer.save_pretrained("./outputs")
config.save_pretrained("./outputs")